# PFG: Streaming API

Se realizan consultas en el flujo global de consultas de Twitter. No es histórico, sino en tiempo real.

## Carga inicial de datos

In [1]:
import nltk
nltk.download('stopwords')
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests_oauthlib import OAuth1
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
from pymongo import MongoClient
import simplejson as json
consumer_key='ynSB0dFvqPl3xRU7AmYk39rGT'
consumer_secret='6alIXTKSxf0RE57QK3fDQ8dxdvlsVr1IRsHDZmoSlMx96YKBFD'
access_token='966591013182722049-BVXW14Hf5s6O2oIwS3vtJ3S3dOsKLbY'
access_token_secret='829DTKPjmwsSytmp1ky9fMCJkjV0LZ04TbL9oqHGV6cDm'

[nltk_data] Downloading package stopwords to C:\Users\Servicio
[nltk_data]     Técnico\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
url = 'https://stream.Twitter.com/1.1/statuses/filter.json'
auth = OAuth1(consumer_key, consumer_secret, access_token, access_token_secret)
pms = {'track' : 'premier league -filter:retweets AND -filter:replies', 'lang': 'en'}

Como con el REST_API se debe realizar la configuración de la información que se quiere rescatar en el Streaming, cargando la información necesaria del endpoint, las librerías, las claves y los tokens de acceso y los parámetros necesarios. 
La diferencia fundamental en la variable **pms** (la de los parámetros) es que, en este caso, al contrario que en REST_API, se indica como primer parámetro track y en REST_API se indicaba una consulta q.

In [4]:
database_name='streamingData'
collection_name='streamingColeccion'
client = MongoClient('mongodb://localhost:27017/')
db = client[database_name]
mongo = db[collection_name] #en REST_API el nombre de la variable era collection, pero aquí lo cambio por mongo para que coincida con la sección de código siguiente

<font color='red'>Para poder insertar la información en la base de datos se debe importar de nuevo la librería que permite utilizar Mongo. Seguramente no será igual que en el REST_API, pero lo pongo por ahora para después modificarlo.</font> Según la [documentación de mongo](https://docs.mongodb.com/manual/reference/method/db.collection.insert/) el método insert se utiliza en una colección de la base de datos, así que cambio el nombre de la variable collection por mongo.<font color='red'>falta establecer el nombre de la base de datos y el nombre de la colección</font>

In [ ]:
res = requests.post(url, auth=auth, params = pms, stream = True)
texto = 'entra'
for line in res.iter_lines(): 
    if line:
        tweet = json.loads(line.decode('utf-8'))
        try:
            mongo.insert(tweet)
        except:
            pass

Esta sección de código se encarga de recibir la información.
Para ello en primer lugar se almacena en la variable **res** la información necesaria para la *consulta*: el endpoint, la información de autorización los parámetros e indicamos con una variable booleana que se trata de un Stream. 
Todo esto se hace mediante una petición post.
A continuación, mediante un bucle for se va iterando sobre cada línea que vaya devolviendo el post y por cada iteración se transforma la información en json (ya que viene en binario) y a continuación se intenta almacenar en la base de datos.

In [ ]:
documents = []
for doc in collection.find():
    documents.append(doc)
df = bd.DataFrame(documents) #en el libro ponía pd, se tratará de una errata

Lo que se hace a continuación es utilizar la base de datos de mongo en la que se ha estado almacenando los tweets para volcar la información de forma tabular, donde las columnas indican los nombres de los nodos del documento y las filas representan los datos.
En **df** se guarda el DataFrame extraído previamente en **documents**.

In [ ]:
df = pd.DataFrame(documents)
df['tweet_source'] = df['source'].apply(lambda x:BeautifulSoup(x).get_text())
devices = list(set(df[df['tweet_source'].str.startswith('Twitter')]['tweet_source']))
devices.remove('Twitter Ads')
df = df[df['tweet_source'].isin(devices)]
#TOKENIZATION
df['tokens'] = df['text'].apply(TweetTokenizer().tokenize)
#STOPWORDS
stopwords_vocabulary = stopwords.words('spanish') #estará en español?
df['stopwords'] = df['tokens'].apply(lambda x: [i for i in x if i.lower() not in stopwords_vocabulary])
#SPECIAL CHARACTERS AND STOPWORDS REMOVAL
punctuations = list(string.punctuation)
df['punctuation'] = df['stopwords'].apply(lambda x: [i for i in x if i not in punctuations])
df['digits'] = df['punctuation'].apply(lambda x: [i for i in x if i[0] not in list(string.digits)])
df['final'] = df['digits'].apply(lambda x: [i for i in x if len(i) > 1])